# Igual que ayer (kmeans) pero sin smaller, con todo el fichero en spark

In [1]:
data = sc.textFile('data/sales_segments.csv.gz')

In [2]:
data.map(lambda l: l.split('^')[51]).take(3)

[u'revenue_amount_seg', u'33.56', u'306.00']

In [3]:
header = data.first()
header

u'rloc^departure_date_seg^booking_date^freq_flyer_flag^creator_office_id^creator_iata_number^creator_gds_code^creator_city_code^creator_lat^creator_lon^creator_country_code^creator_continent_code^creator_channel_code^booking_status_code^cabin_code^booking_class_code^pos_office_id^pos_iata_number^pos_gds_code^pos_city_code^pos_lat^pos_lon^pos_country_code^pos_continent_code^pos_channel_code^board_point_seg^board_lat^board_lon^board_country_code^board_continent_code^off_point_seg^off_lat^off_lon^off_country_code^off_continent_code^num_legs^distance_seg^route^network^airline_code^flight_number^flight_type^revenue_source^estimated_flag^usage_board_point_seg^usage_off_point_seg^usage_flight_number^usage_date^non_currency_type^bookings_seg^rpk_seg^revenue_amount_seg^currency_code^fuel_surcharge_amount_seg^fuel_surcharge_currency_code^emd_count^emd_amount^emd_currency_code^total_outliers'

In [4]:
# Necesito quitar la primera fila del header con filter
# estoy en big data no vale quitar la primera línea pues no sé en que trozo funciono)

lines = data.filter(lambda line: line != header)

In [5]:
fields = ['bookings_seg','revenue_amount_seg','fuel_surcharge_amount_seg']
[header.split('^').index(field) for field in fields]

[49, 51, 53]

In [6]:
def average_fare_tax(line):
    elems = line.split('^')
    bookings = float(elems[49])
    rev = float(elems[51])
    tax = float(elems[53])
    av_rev = rev / bookings
    av_tax = tax / bookings
    return (av_rev,av_tax)
average_fare_tax(lines.first())

(33.56, 7.3)

In [7]:
simple = lines.map(average_fare_tax)
simple.take(3)

[(33.56, 7.3), (61.2, 25.5), (71.86842105263158, 12.31578947368421)]

In [8]:
# Ahora importamos kmeans pero en Spark (mirar spark.apache.org)
# http://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=kmeans#pyspark.ml.clustering.KMeans

from pyspark.mllib.clustering import KMeans, KMeansModel

In [9]:
clusters = KMeans.train(simple,10)

In [10]:
clusters.save(sc,'clusters')

In [11]:
clusters.centers

[array([ 118.67860863,   69.76914167]),
 array([  8.77500428e+05,   3.03020000e+01]),
 array([  4.50012295e+05,   1.28675000e+02]),
 array([  1.06350097e+05,   2.19353061e+01]),
 array([  3.39158820e+05,   8.87454545e+01]),
 array([  1.39215574e+06,   3.27400000e+01]),
 array([  6.29082976e+05,   2.97314286e+01]),
 array([  2.35943244e+05,   5.44940000e+01]),
 array([ 480.45523153,  190.18995911]),
 array([ 1845.75904504,   207.21854771])]

In [13]:
clusters.predict((33.56,7.3))

0

In [16]:
# IMPOSIBLE HACERLO CON GROUPBYKEY
bycluster = simple.map(lambda point: (clusters.predict(point), point)).cache()
bycluster.take(3)

[(0, (33.56, 7.3)),
 (0, (61.2, 25.5)),
 (0, (71.86842105263158, 12.31578947368421))]

In [21]:
bycluster_withones = simple.map(lambda (x,y): (clusters.predict((x,y)),(x,y,1)))
bycluster_withones.take(3)

[(0, (33.56, 7.3, 1)),
 (0, (61.2, 25.5, 1)),
 (0, (71.86842105263158, 12.31578947368421, 1))]

In [25]:
def calc_avg_reduce(acc,p):
    (rev_p,tax_p,c) = p
    (rev_ac, tax_ac, c_acc) = acc
    result = (rev_p + rev_ac, tax_p + tax_ac,  c +  c_acc)
    return result

In [26]:
avs = bycluster_withones.reduceByKey(calc_avg_reduce)
avs.take(3)

[(0, (136860764.86963308, 80458123.02322179, 1153205)),
 (1, (8775004.28, 303.02, 10)),
 (2, (1800049.18, 514.7, 4))]

In [32]:
def medias(clust):
    rev_t, tax_t, count = clust
    return (rev_t / count, tax_t / count, count)

averages = avs.mapValues(medias)
averages.take(10)

[(0, (118.67860863387956, 69.76914167318195, 1153205)),
 (1, (877500.428, 30.302, 10)),
 (2, (450012.295, 128.675, 4)),
 (3, (106350.09734693861, 21.93530612244898, 98)),
 (4, (339158.82, 88.74545454545454, 11)),
 (5, (1392155.74, 32.74, 1)),
 (6, (629082.9757142857, 29.73142857142857, 7)),
 (7, (235943.244, 54.49400000000001, 5)),
 (8, (480.4552315300932, 190.18995910799475, 290796)),
 (9, (1845.75904503611, 207.21854771412237, 14984))]

In [33]:
# Lo siguiente está mal, identifica revenue = 0 con cluster = 0
simple.join(averages).take(3)


[(0.0, (15.91, (118.67860863387956, 69.76914167318195, 1153205))),
 (0.0, (15.76, (118.67860863387956, 69.76914167318195, 1153205))),
 (0.0, (7.41, (118.67860863387956, 69.76914167318195, 1153205)))]

In [34]:
bycluster_withones.join(averages).take(3)

[(0, ((33.56, 7.3, 1), (118.67860863387956, 69.76914167318195, 1153205))),
 (0, ((61.2, 25.5, 1), (118.67860863387956, 69.76914167318195, 1153205))),
 (0,
  ((71.86842105263158, 12.31578947368421, 1),
   (118.67860863387956, 69.76914167318195, 1153205)))]

In [38]:
def sq_diffs(p_with_cluster_mean):
    ((rev,tax,n),(av_rev,av_tax,n_cl)) = p_with_cluster_mean
    rev_sqdiff = (rev-av_rev) ** 2
    tax_sqdiff = (tax-av_tax) ** 2
    return (rev_sqdiff, tax_sqdiff,1)

In [39]:
points_w_means_sqdiffs  = bycluster_withones.join(averages).mapValues(sq_diffs)
points_w_means_sqdiffs.take(3)

[(0, (7245.177535767556, 3902.393661384078, 1)),
 (0, (3303.7904504866938, 1959.7569044802547, 1)),
 (0, (2191.193661391623, 3300.8876789595315, 1))]

In [41]:
total_sqdiffs = points_w_means_sqdiffs.reduceByKey(calc_avg_reduce)
total_sqdiffs.take(3)

[(0, (7591139774.465887, 6412116670.814349, 1153205)),
 (8, (8884941916.585518, 927167489.812048, 290796)),
 (2, (8746772217.281904, 117267.7809, 4))]

In [47]:
from math import sqrt
def stdevs(sqdiffs_percluster):
    (sqd_rev_total, sqd_tax_total, n_cl) = sqdiffs_percluster   
    std_rev = sqrt(sqd_rev_total / n_cl)
    std_tax = sqrt(sqd_tax_total / n_cl)
    return(std_rev,std_tax)

stds_clusters = total_sqdiffs.mapValues(stdevs)
stds_clusters.take(12)

[(0, (81.13350455424671, 74.56713371911901)),
 (8, (174.7966408531226, 56.46572170857911)),
 (2, (46762.08992678231, 171.22191806249572)),
 (4, (23365.27515505823, 77.68410395062537)),
 (6, (35870.99622630394, 1.542099020087588)),
 (1, (47658.199670890375, 1.7160000000000006)),
 (3, (15537.311648079669, 64.2896395500684)),
 (9, (2263.534061018395, 63.655635286580804)),
 (5, (0.0, 0.0)),
 (7, (25966.61177423162, 49.39311069369898))]

In [49]:
cluster_stats = averages.join(stds_clusters)
cluster_stats.collect()

[(0,
  ((118.67860863387956, 69.76914167318195, 1153205),
   (81.13350455424671, 74.56713371911901))),
 (9,
  ((1845.75904503611, 207.21854771412237, 14984),
   (2263.534061018395, 63.655635286580804))),
 (3,
  ((106350.09734693861, 21.93530612244898, 98),
   (15537.311648079669, 64.2896395500684))),
 (6,
  ((629082.9757142857, 29.73142857142857, 7),
   (35870.99622630394, 1.542099020087588))),
 (1, ((877500.428, 30.302, 10), (47658.199670890375, 1.7160000000000006))),
 (4,
  ((339158.82, 88.74545454545454, 11),
   (23365.27515505823, 77.68410395062537))),
 (7,
  ((235943.244, 54.49400000000001, 5),
   (25966.61177423162, 49.39311069369898))),
 (8,
  ((480.4552315300932, 190.18995910799475, 290796),
   (174.7966408531226, 56.46572170857911))),
 (2, ((450012.295, 128.675, 4), (46762.08992678231, 171.22191806249572))),
 (5, ((1392155.74, 32.74, 1), (0.0, 0.0)))]

In [51]:
full = bycluster.join(cluster_stats)
full.take(3)

[(0,
  ((33.56, 7.3),
   ((118.67860863387956, 69.76914167318195, 1153205),
    (81.13350455424671, 74.56713371911901)))),
 (0,
  ((61.2, 25.5),
   ((118.67860863387956, 69.76914167318195, 1153205),
    (81.13350455424671, 74.56713371911901)))),
 (0,
  ((71.86842105263158, 12.31578947368421),
   ((118.67860863387956, 69.76914167318195, 1153205),
    (81.13350455424671, 74.56713371911901))))]

In [66]:
def zscore(p_w_stats):
    
    p, stats = p_w_stats
    
    rev,tax = p
    means, stdevs = stats
    rev_m, tax_m, _ = means     # por convención _ indica que no se utilizará ese tercer elemento
    rev_std, tax_std = stdevs
    
    if rev_std == 0:
        return rev, tax, 3000, True  # Es un poco guarrería, pero evita un error posterior cuando hay un solo nodo en un cluster
    if tax_std == 0:
        return rev, tax, 3000, True  # Es un poco guarrería, pero evita un error posterior cuando hay un solo nodo en un cluster
     
    zscore_rev = (rev-rev_m) / rev_std
    zscore_tax = (tax-tax_m) / tax_std
    
    zscore = sqrt(zscore_rev ** 2 + zscore_tax ** 2)
    outlier = zscore > 3
    
    return rev, tax, zscore, outlier

zscore(full.first()[1])

(33.56, 7.3, 1.3425666282536965, False)

In [67]:
outliers = full.mapValues(zscore)
outliers.take(3)

[(0, (33.56, 7.3, 1.3425666282536965, False)),
 (0, (61.2, 25.5, 0.9243116171177309, False)),
 (0, (71.86842105263158, 12.31578947368421, 0.9625651495519475, False))]

In [69]:
# Hasta aqui no se ha ejecutado nada (SE DEFINE UNA DESCRIPCIÓN DE UNA EJECUCIÓN). Concepto outlier es un RDD.
outliers.saveAsTextFile('outliers')